In [ ]:
%pip install -q onnx

In [12]:
import torch
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn

In [13]:
class BERTRegressor(nn.Module):
    def __init__(self, base_model):
        super().__init__()

        self.bert = base_model

        self.regressor = nn.Sequential(
             nn.Dropout(0.5),
             nn.Linear(self.bert.config.hidden_size, 128),
             nn.GELU(),
             nn.Linear(128, 1),
         )

    def forward(self, input_ids, attention_mask):
        cls = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0]

        return self.regressor(cls).squeeze(-1)

In [14]:
# Load base model and custom model
base_model = AutoModel.from_pretrained("answerdotai/ModernBERT-base")
model = BERTRegressor(base_model)
model.load_state_dict(torch.load("/content/drive/MyDrive/BERT_rating_prediction/model/weights.pth"))
model.eval()

# ONNX export doesn't require GPU
model.to("cpu")

# Prepare dummy input
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
inputs = tokenizer("This is a dummy input.", return_tensors="pt", padding=True, truncation=True)

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Export to ONNX
torch.onnx.export(
    model,
    (input_ids, attention_mask),
    "/content/drive/MyDrive/BERT_rating_prediction/model/bert_regressor.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["output"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "seq_len"},
        "attention_mask": {0: "batch_size", 1: "seq_len"},
        "output": {0: "batch_size"}
    },
    opset_version=17
)